In [3]:
from typing import Any, Dict, List

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.teams import Swarm
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [4]:
from dotenv import load_dotenv
load_dotenv()
import os 
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key, parallel_tool_calls=False)


In [5]:
def refund_flight(flight_PNR:str)-> str:
    return f"Refunded Flight with PNR {flight_PNR}"

In [6]:
travel_agent =AssistantAgent(
        name='travel_agent',
        model_client=model_client,
        handoffs=['flights_refunder','user'],
        system_message="""You are a travel agent.
    The flights_refunder is in charge of refunding flights.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    Use TERMINATE when the travel planning is complete.""",

)

In [7]:
flights_refunder = AssistantAgent(
    name='flights_refunder',
    model_client=model_client,
    handoffs=['travel_agent', 'user'],
    tools=[refund_flight],
    system_message="""You are an agent specialized in refunding flights.
    You only need flight PNR numbers to refund a flight.
    You have the ability to refund a flight using the refund_flight tool.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    When the transaction is complete, handoff to the travel agent to finalize.""",
)


In [8]:
termination = HandoffTermination(target='user') | TextMentionTermination('TERMINATE')
team = Swarm(participants=[travel_agent,flights_refunder], termination_condition= termination)


In [9]:
await team.reset()

In [10]:
task = " i want to refund my flight"

async def run_team_stream()-> None:
    task_result = await Console(team.run_stream(task=task)
                                )
    
    last_message = task_result.messages[-1]

    while (isinstance(last_message, HandoffMessage) and last_message.target =='user'):
        user_input = input("User : ")

        task_result = await Console(team.run_stream(task=HandoffMessage(source='user',target=last_message.source, content=user_input))
                                    )
        last_message = task_result.messages[-1]
        
await run_team_stream()



---------- TextMessage (user) ----------
 i want to refund my flight
---------- ThoughtEvent (travel_agent) ----------
I can assist you with that. Let me connect you to the flights_refunder who will help process your flight refund. Please hold on a moment.
---------- ToolCallRequestEvent (travel_agent) ----------
[FunctionCall(id='call_nMEgRkWLY03g1BGK3XqE4I6h', arguments='{}', name='transfer_to_flights_refunder')]
---------- ToolCallExecutionEvent (travel_agent) ----------
[FunctionExecutionResult(content='Transferred to flights_refunder, adopting the role of flights_refunder immediately.', name='transfer_to_flights_refunder', call_id='call_nMEgRkWLY03g1BGK3XqE4I6h', is_error=False)]
---------- HandoffMessage (travel_agent) ----------
Transferred to flights_refunder, adopting the role of flights_refunder immediately.
---------- TextMessage (flights_refunder) ----------
To proceed with the refund, could you please provide me with your flight's PNR number?
---------- ToolCallRequestEven